In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split


In [2]:
def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.endswith(".png"):
            img = cv2.imread(os.path.join(folder_path, filename))
            if img is not None:
                images.append(img)
    return images

# Path to your image folder
image_folder = '/Users/nishchayaroy/Desktop/ML_final_sub/oath_v1.1/images/cropped_scaled'

# Load the color images
color_images = load_images_from_folder(image_folder)

In [3]:
def preprocess_images(color_images):
    X = []  # Grayscale input images
    Y = []  # a* and b* target channels
    
    for img in color_images:
        # Convert to Lab color space
        lab_image = cv2.cvtColor(img, cv2.COLOR_BGR2Lab)
        
        # Extract the L (grayscale) channel
        L_channel = lab_image[:, :, 0]
        
        # Extract the a* and b* channels (target color channels)
        ab_channels = lab_image[:, :, 1:3]
        
        # Normalize the grayscale and color channels to [0, 1]
        L_channel = L_channel.astype('float32') / 255.0
        ab_channels = ab_channels.astype('float32') / 255.0
        
        X.append(L_channel)
        Y.append(ab_channels)
    
    # Convert to numpy arrays
    X = np.array(X)
    Y = np.array(Y)
    
    # Add channel dimension for grayscale images (single-channel input)
    X = np.expand_dims(X, axis=-1)
    
    return X, Y

# Preprocess the images
X, Y = preprocess_images(color_images)

# Split the dataset into training and validation sets
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)


In [4]:
def build_colorization_model(input_shape=(256, 256, 1)):
    inputs = layers.Input(shape=input_shape)

    # Encoder (downsampling)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)

    # Decoder (upsampling)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)

    # Output layer (colorization)
    outputs = layers.Conv2D(2, (3, 3), activation='tanh', padding='same')(x)

    # Define the model
    model = models.Model(inputs, outputs)
    return model

# Create the model
model = build_colorization_model()

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Model summary
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 256, 256, 1)]     0         
                                                                 
 conv2d (Conv2D)             (None, 256, 256, 64)      640       
                                                                 
 max_pooling2d (MaxPooling2  (None, 128, 128, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 128, 128, 128)     73856     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 64, 64, 128)       0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 256)       295168

In [5]:
history = model.fit(
    X_train, Y_train, 
    batch_size=32, 
    epochs=20, 
    validation_data=(X_val, Y_val)
)


In [6]:
# Plotting the training and validation loss
import matplotlib.pyplot as plt

# Access training and validation loss
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plot the loss curves
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training vs Validation Loss')
plt.show()


NameError: name 'history' is not defined

In [ ]:
# Evaluate the model on the validation data
val_loss = model.evaluate(X_val, Y_val)
print(f"Validation Loss: {val_loss}")


In [ ]:
history = model.fit(
    X_train, Y_train, 
    batch_size=32, 
    epochs=20, 
    validation_data=(X_val, Y_val)
)

# Plotting MAE along with the loss
mae_train = history.history['mean_absolute_error']
mae_val = history.history['val_mean_absolute_error']

plt.plot(mae_train, label='Training MAE')
plt.plot(mae_val, label='Validation MAE')
plt.xlabel('Epochs')
plt.ylabel('Mean Absolute Error')
plt.legend()
plt.title('Training vs Validation MAE')
plt.show()
